In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('order_brush_order.csv')

In [ ]:
data

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [ ]:
data = data.sort_values(by=['shopid','event_time'])
data = data.reset_index(drop=True)
data

,orderid,shopid,userid,event_time
0,31086409141107,10009,196962305,2019-12-27 03:06:50
1,31144571933461,10051,2854032,2019-12-27 19:16:11
2,31254979546679,10051,48600461,2019-12-29 01:56:19
3,31195675919209,10061,168750452,2019-12-28 09:27:55
4,31205132327893,10061,194819216,2019-12-28 12:05:32
...,...,...,...,...
222745,31493203163305,214949521,46269178,2019-12-31 20:06:43
222746,31328775676314,214964814,200983383,2019-12-29 22:26:16
222747,31453591917585,215175775,13688804,2019-12-31 09:06:31
222748,31472077221692,215175775,129266028,2019-12-31 14:14:37


In [ ]:
data['event_time'] = pd.to_datetime(data['event_time'])

In [ ]:
data.dtypes

orderid                int64
shopid                 int64
userid                 int64
event_time    datetime64[ns]
dtype: object

In [ ]:
sejam = pd.Timedelta('1 hour')

In [ ]:
sejam

Timedelta('0 days 01:00:00')

In [ ]:
(data.iloc[1].event_time-data.iloc[0].event_time) >= sejam

True

In [ ]:
import operator
ans = dict()
preShop = data.iloc[0].shopid
ans[preShop] = []
p1 = 0
p2 = 0
user = dict()
eps = 0.00000001

for id,row in data.iterrows():

  nowShop = row['shopid']
  nowUser = row['userid']
  nowTime = row['event_time']

  if(preShop != nowShop): #ganti shop   

    #cari rate
    uniqueUser = len(user)
    totalTrans = p2-p1+1
    rate = totalTrans/uniqueUser

    if(rate+eps >= 3.000):
      #cari buyer suspect
      itemMaxValue = max(user.items(), key=lambda x: x[1])
      for key, value in user.items():
          if value == itemMaxValue[1]:
              ans[preShop].append(key)

    #reset
    ans[nowShop] = []
    p1 = id
    p2 = id
    user.clear()
    user[nowUser] = 1
    print(len(user))

  else: #masih dalam shop yg sama

    tmpTime = data.iloc[p1].event_time

    if((nowTime-tmpTime) <= sejam): #tambah ke interval

      #masukin p2
      if(nowUser in user): user[nowUser] += 1
      else: user[nowUser] = 1
      p2 = id

    else: #g masuk, geser p1
  
      while 1: #geser p1
        if( (nowTime-tmpTime) <= sejam ): break

        uniqueUser = len(user)
        totalTrans = p2-p1+1
        rate = totalTrans/uniqueUser

        if(rate+eps >= 3.000):
          #cari buyer suspect
          itemMaxValue = max(user.items(), key=lambda x: x[1])
          for key, value in user.items():
              if value == itemMaxValue[1]:
                  ans[nowShop].append(key)
        tmpUser = data.iloc[p1].userid
        user[tmpUser] -= 1
        if user[tmpUser] == 0 : user.pop(tmpUser)
        p1 +=1
        tmpTime = data.iloc[p1].event_time

      #masukin p2
      if(nowUser in user): user[nowUser] += 1
      else: user[nowUser] = 1
      p2 = id

  preShop = nowShop
  

In [ ]:
#cari rate
uniqueUser = len(user)
totalTrans = p2-p1+1
rate = totalTrans/uniqueUser

if(rate+eps >= 3.000):
  #cari buyer suspect
  itemMaxValue = max(user.items(), key=lambda x: x[1])
  for key, value in user.items():
      if value == itemMaxValue[1]:
          ans[nowShop].append(key)

In [ ]:
formatAns = dict()
shop = []
buyer = []

for nowshop in ans:
  tmp = ""
  first = 1
  ans[nowshop].sort()
  for user in ans[nowshop]:
    if first==1 : 
      tmp = str(user)
      first = 0
    else:
      tmp = tmp + "&" + str(user) 
  if tmp == "" : tmp = '0'
  shop.append(nowshop)
  buyer.append(tmp)

formatAns['shopid'] = shop
formatAns['userid'] = buyer

df = pd.DataFrame.from_dict(formatAns)

In [ ]:
df

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
...,...,...
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0


In [ ]:
from google.colab import files
df.to_csv('Olea.csv', index=False) 
files.download('Olea.csv')